**[필요한 라이브러리 호출 및 API키 설정]**

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables.history import BaseChatMessageHistory, RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate

c:\Users\user\Desktop\MODULABS\LangchainThon\LangchainThon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **[데이터 로드]**

**[청크 분할]**

In [3]:
# Chunk split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]             
)

**[PDF 문서 로드/분할 및 벡터 임베딩]**

In [4]:
from langchain_community.document_loaders import PyPDFLoader

PDF_PATH = r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\pdf\모두연 브랜딩북 정리.pdf"

loader_pdf = PyPDFLoader(PDF_PATH)
pages_pdf = loader_pdf.load()

docs_pdf = text_splitter.split_documents(pages_pdf)

**[HTML 문서 로드/분할 및 벡터 임베딩]**

In [5]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

HTML_PATH = [
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\출결 및 공가에 대하여 23f2d25db6248149a6faf3763fbcfdae.html",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\html\데싸 5기 훈련 정보 23f2d25db62481f3aa6bc6dbdf2fc11c.html"
]

html_list = []

# 각 파일 로드
for path in HTML_PATH:
    loader_html = UnstructuredHTMLLoader(path)
    pages_html = loader_html.load()
    html_list.extend(pages_html)  

docs_html = text_splitter.split_documents(html_list)

**[WORD 문서 로드/분할 및 벡터 임베딩]**

In [6]:
from langchain_community.document_loaders import Docx2txtLoader

WORD_PATH = r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\word\휴가신청서(데싸_5기).docx"
loader_word = Docx2txtLoader(WORD_PATH)
pages_word = loader_word.load()

docs_word = text_splitter.split_documents(pages_word)

**[CSV 문서 로드/분할 및 벡터 임베딩]**

In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader

CSV_PATH = [
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데이터 사이언티스트 5기 일정표.csv",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데싸 5기 동료들.csv",
    r"C:\Users\user\Desktop\MODULABS\LangchainThon\Data\csv\데싸 5기 운영진.csv"
]

csv_list = []

# 각 파일 로드
for path in CSV_PATH:
    loader_csv = CSVLoader(path, encoding = 'cp949')
    pages_csv = loader_csv.load()
    csv_list.extend(pages_csv)  

docs_csv = text_splitter.split_documents(csv_list)

In [8]:
len(docs_csv)

142

In [23]:
print(docs_csv[10].page_content)

이름: 한현종 휴가
전산신청(운매체크): No
Files & media: 
Text: 
공결여부(그루체크): Yes
날짜: 03/14/2024
부재시간: 
비고 (메모): 
상태: 출결
서류제출: No


**[벡터db 저장]**

In [10]:
# ChromaDB 벡터 임베딩 후 저장
vectorstore = Chroma.from_documents(docs_html, OpenAIEmbeddings(model='text-embedding-3-small'))

vectorstore.add_documents(docs_word)
vectorstore.add_documents(docs_csv)
vectorstore.add_documents(docs_pdf)

retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

## **[RAG 시스템]**

**[채팅 히스토리와 사용자 질문 통합]**

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# LLM 모델 선언
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 질문 프롬프트
contextualize_q_system_prompt = """이전 대화가 있다면 참고하여,
사용자의 최신 질문을 독립적으로 이해 가능한 한 문장으로 바꿔주세요.
답변하지 말고 질문만 재작성하세요."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, 
    retriever, 
    contextualize_q_prompt
)

# 답변 프롬프트
qa_system_prompt = """아래 문서 내용만을 근거로 답하세요.
근거가 없으면 '문서에서 관련 내용이 확인되지 않습니다.'라고 대답하세요.
최대 3문장으로 짧게 답변하세요.

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

**[RAG 체인 구축]**

In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

**[세션별 기록 저장]**

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

#채팅 세션별 기록 저장 위한 Dictionary 선언
store = {}

#주어진 session_id 값에 매칭되는 채팅 히스토리 가져오는 함수 선언
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


#RunnableWithMessageHistory 모듈로 rag_chain에 채팅 기록 세션별로 자동 저장 기능 추가
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

### **[테스트]**

In [14]:
# ===================
# 테스트 1
# ===================

#채팅 히스토리를 적재하기 위한 리스트
chat_history = []

question = "몇 시부터 지각이야?"

#첫 질문에 답변하기 위한 rag_chain 실행
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

#첫 질문과 답변을 채팅 히스토리로 저장
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "조퇴는?"

#두번째 질문 입력 시에는 첫번째 질문-답변이 저장된 chat_history가 삽입됨
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

조퇴는 14:45부터 시작된 경우가 있으며, 다른 경우에도 15시 이후에 조퇴가 발생한 기록이 있습니다.


In [15]:
# ===================
# 테스트 2
# ===================

chat_history = []

question = "몇 시부터 지각이야?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "휴가 신청서 작성해야 한다고 들었는데, 어떤 서류야?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

문서에서 관련 내용이 확인되지 않습니다.


In [16]:
# ===================
# 테스트 3
# ===================

question = "휴가쓰고 싶어. 휴가 신청하려면 어떻게 해야해?"
result = rag_chain.invoke({"input": question, "chat_history": []})

print(result["answer"])

문서에서 관련 내용이 확인되지 않습니다.


In [17]:
# ===================
# 테스트 4
# ===================

chat_history = []

question = "데이터 사이언스 5기 동료들 중 MBTI가 ISTJ 인 사람은 누구야?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1["answer"])])

second_question = "그 사람들의 취미는 뭐야?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

김정인의 취미는 맥켄지 신전운동, 정주이의 취미는 여행과 캠핑, 손호진의 취미는 여행과 외국어 공부, 이종석의 취미는 음악 감상과 게임, 차정은의 취미는 확인되지 않으며, 홍성민의 취미는 게임입니다.


In [18]:
# ===================
# 테스트 5
# ===================

question = "나는 데싸5기 손호진이야. 지금까지 내가 총 조퇴와 결석을 몇 번 했는지 궁금해."
result = rag_chain.invoke({"input": question, "chat_history": []})

print(result["answer"])

문서에서 관련 내용이 확인되지 않습니다.


In [19]:
# ===================
# 테스트 6
# ===================

question = "모두연이 뭐야?"
result = rag_chain.invoke({"input": question, "chat_history": []})

print(result["answer"])

모두연은 '모두의 연구소'로, 자유롭게 연구하고 학습할 수 있는 커뮤니티입니다. 이곳은 경쟁 없이 협력하며 상생과 성장을 추구하는 공간으로, 스스로 학습할 수 있는 환경을 제공합니다. 또한, 다양한 주제를 탐구하고 함께 배우는 것을 중요시합니다.


In [20]:
# ===================
# 테스트 7
# ===================

question = "모두의 연구소의 핵심 슬로건이 뭐야?"
result = rag_chain.invoke({"input": question, "chat_history": []})

print(result["answer"])

모두의 연구소의 핵심 슬로건은 ‘쉐벨그투’로, 이는 쉐어 벨류 그로우 투게더(SHARE VALUE, GROW TOGETHER)의 각 영어 단어의 첫 한글 발음 모음입니다.


In [21]:
# ===================
# 테스트 8
# ===================

question = "쉐벨그투에 대해 알려줘."
result = rag_chain.invoke({"input": question, "chat_history": []})

print(result["answer"])

쉐벨그투는 '쉐어 벨류 그로우 투게더(SHARE VALUE, GROW TOGETHER)'의 약자로, 각 영어 단어의 첫 한글 발음 모음으로 구성된 핵심 슬로건입니다.
